In [ ]:
import numpy as np
import re
# 패키지 설치
import pandas as pd
#warning 메시지 표시 안함
import warnings
warnings.filterwarnings(action = 'ignore')
from konlpy.tag import Okt # 형태소 분석에 사용할 konlpy 패키지의 Okt 클래스를 임포트하고 okt 객체를 생성
okt = Okt()
from konlpy.tag import Kkma, Komoran, Okt, Hannanum
from collections import Counter
# Train 데이터 불러오기
train_df = pd.read_csv('/content/Lyrics_v2.csv')

# 데이터 확인
print(train_df.head())

                                              Lyrics  label
0  아침에 일어나 너에게 짧은 인사를 보낸다 아무리 멀리 떨어져 있어도 나는 널 생각하...      0
1  라라랄라 라라랄라 이어 이어 네가 맛있었다고 했던 살구빛 샐러드 맛있어서 또 먹고 ...      0
2  자꾸 마주치는 네 눈빛에 왠지 어색해 대체 왜 이래 고개를 숙여봐도 너의 발끝에 맘...      0
3  어깨동무까진 괜찮아 그 치마 손잡는 건 조금 위험해 잘 자란 메세 진 괜찮아그래도 ...      0
4  햇살이 가득 그댈 향한 가벼운 발걸음 콧노래 부르며 쇼윈도에 비친 내 모습을 한번 ...      0


In [ ]:
text = train_df['Lyrics']
score = train_df['label']

In [ ]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(text, score , test_size=0.2, random_state=0)
print(len(train_x), len(train_y), len(test_x), len(test_y))

160 160 40 40


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfv = TfidfVectorizer(tokenizer=okt.morphs, ngram_range=(1,2), min_df=3, max_df=0.9)
tfv.fit(train_x)
tfv_train_x = tfv.transform(train_x)
print(tfv_train_x)

In [ ]:
from sklearn.linear_model import LogisticRegression # 이진 분류 알고리즘
from sklearn.model_selection import GridSearchCV # 하이퍼 파라미터 최적화

clf = LogisticRegression(random_state=0)
params = {'C': [15, 18, 19, 20, 22]}
grid_cv = GridSearchCV(clf, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv.fit(tfv_train_x, train_y)

# 최적의 평가 파라미터는 grid_cv.best_estimator_에 저장됨
print(grid_cv.best_params_, grid_cv.best_score_)# 가장 적합한 파라메터, 최고 정확도 확인

Fitting 3 folds for each of 5 candidates, totalling 15 fits
{'C': 15} 0.5627766130910784


In [ ]:
tfv_test_x = tfv.transform(test_x)
# test_predict = grid_cv.best_estimator_.score(tfv_test_x,test_y)
test_predict = grid_cv.best_estimator_.predict(tfv_test_x)
from sklearn.metrics import accuracy_score
print('감성 분류 모델의 정확도 : ',round(accuracy_score(test_y, test_predict), 3))

감성 분류 모델의 정확도 :  0.7


In [ ]:
input_text = train_df['Lyrics'][50]
#입력 텍스트에 대한 전처리 수행
input_text = re.compile(r'[ㄱ-ㅣ가-힣]+').findall(input_text)
input_text = [" ".join(input_text)]
# 입력 텍스트의 피처 벡터화
st_tfidf = tfv.transform(input_text)

# 최적 감성 분석 모델에 적용하여 감성 분석 평가
st_predict = grid_cv.best_estimator_.predict(st_tfidf)

#예측 결과 출력
if(st_predict == 0):
    print('예측 결과: ->> 부정 감성')
else:
    print('예측 결과: ->> 긍정 감성')

예측 결과: ->> 긍정 감성


In [ ]:
train_df['Lyrics'][53]

'요즘 너무나 네가 낯설게 느껴져 어제는 전화도 한 번 없었지 어디니 밥은 먹었냐고 묻는 상냥한 네 말 기다렸는데 아무 말도 없는 너를 보며 느껴 내가 알던 네가 맞는 건지 다른 사람 네게 생긴 건지 이제 더 이상 날 사랑하지 않니 우리 이별인 거니 왜 내가 싫어졌는지 내게 말해줄 순 없니 자꾸만 전화해서 귀찮아진 거니 아직도 난 너를 사랑하고 있는데 난 어떻게 난 어떡해 내가 뭘 잘못한 건지 왜 내가 싫증 나는지 뭘 내가 잘못하고 있는 거라면 말해줄래 네가 말하는 그 모습처럼 고칠 거다 고칠게 떠나지 마나는 아직도 네가 내 것만 같은데 오늘도 네게 전화를 걸어서 잘 자라 오늘 밤 꿈에서 보잔 달콤한 말 해줄 것 같은데 왜 내가 싫어졌는지 내게 말해줄 순 없니 자꾸만 전화해서 귀찮아진 거니 아직도 난 너를 사랑하고 있는데 난 어떻게 난 어떡해 내가 뭘 잘못한 건지 왜 내가 싫증 났는지 뭘 내가 잘못하고 있는 거라면 말해줄래 네가 말하는 그 모습처럼 고칠 거다 고칠게 왜 그리 차가운 건지 예전에 네가 맞는지 너 전에 만나던 그 사람 때문이니 말해줄래 네가 만나는 그 사람처럼 고칠 거다 고칠게 떠나지 마 '

In [ ]:
Lyrics = pd.read_csv('/content/Lyrics_v2.csv')
Lyrics

,Lyrics,label
0,아침에 일어나 너에게 짧은 인사를 보낸다 아무리 멀리 떨어져 있어도 나는 널 생각하...,0
1,라라랄라 라라랄라 이어 이어 네가 맛있었다고 했던 살구빛 샐러드 맛있어서 또 먹고 ...,0
2,자꾸 마주치는 네 눈빛에 왠지 어색해 대체 왜 이래 고개를 숙여봐도 너의 발끝에 맘...,0
3,어깨동무까진 괜찮아 그 치마 손잡는 건 조금 위험해 잘 자란 메세 진 괜찮아그래도 ...,0
4,햇살이 가득 그댈 향한 가벼운 발걸음 콧노래 부르며 쇼윈도에 비친 내 모습을 한번 ...,0
...,...,...
195,누구나 손가락질 받을 때면 가슴 한편 이 뜨거워지는 걸 느끼죠 그댄 따뜻한 그 말 ...,2
196,너의 하루가 기나긴 한숨 같아 그저 떠나고 싶은 거니 또 기대하고 휘청이며 버티고 ...,2
197,소년이 어른이 되어 사람을 알아갈 때에 뜻하지 않던 많은 요구와 거친 입술들 소년이...,2
198,왜 한숨이 나는지 왜 고갤 못 드는 건지 나만 이런 건지 무거운 내 발걸음 오늘은 ...,2


In [ ]:
kosac = pd.read_csv("/content/polarity.csv")
kosac

,ngram,freq,COMP,NEG,NEUT,None,POS,max.value,max.prop
0,가*/JKS,1,0.0,0.0,0.0,0.0,1.0,POS,1.0
1,가*/JKS;있/VV,1,0.0,0.0,0.0,0.0,1.0,POS,1.0
2,가*/JKS;있/VV;었/EP,1,0.0,0.0,0.0,0.0,1.0,POS,1.0
3,가*/VV,3,0.0,0.0,0.0,0.0,1.0,POS,1.0
4,가*/VV;ㄴ다*/EF,1,0.0,0.0,0.0,0.0,1.0,POS,1.0
...,...,...,...,...,...,...,...,...,...
16357,힘들/VA;ㄹ/ETM;것/NNB,1,0.0,1.0,0.0,0.0,0.0,NEG,1.0
16358,힘들/VA;ㄹ/ETM;때/NNG,1,0.0,1.0,0.0,0.0,0.0,NEG,1.0
16359,힘차/VA,1,0.0,1.0,0.0,0.0,0.0,NEG,1.0
16360,힘차/VA;ㄴ/ETM,1,0.0,1.0,0.0,0.0,0.0,NEG,1.0


In [ ]:
pos = pd.read_csv('/content/pos_pol_word.txt').values.tolist()
pos

In [ ]:
pos_word = kosac[kosac['max.value'] == 'POS']
neg_word = kosac[kosac['max.value'] == 'NEG']


In [ ]:
text = Lyrics['Lyrics'].to_list()
text[0]

'아침에 일어나 너에게 짧은 인사를 보낸다 아무리 멀리 떨어져 있어도 나는 널 생각하고 있어 늦은 밤 졸린 눈 비비며 너에게 편지를 쓸게 반가운 사진 속 그 표정만큼 아름다운 나날들이기를 찬 바람이 불어와 어깨가 움츠러들 때에도 너를 떠올린다면 씩씩한 표정 할 수 있어 알고 있니 넌 나의 두근대는 맘은 아직까지 여기에 언제까지나 좋아한단 말은 나를 웃음 짓게 해 때로는 지쳐 울어도 우 우 우 우 눈을 감으니 꼭 곁에 있는 것 같아 짠 하고 나타날 게 생각지 못한 어느 날 우연히 문득 뒤돌아 봤을 때 그때 꼭 내가 서 있을게 꼭 하고 껴안을게 가장 반가운 얼굴로 눈물이 왈칵 쏟아진다 해도 나는 환하게 웃을 거야 혹시 오늘 너에게 서러운 일이 생긴다면 오래전 그날처럼 달려가 꼭 안아주고파아름답게 빛나길 알고 있니 넌 나의 두근대는 맘은 아직까지 여기에 언제까지나 좋아한단 말은 나를 웃음 짓게 해 때로는 지쳐 울어도 눈을 감으니 꼭 곁에 있는 것 같아 기억해줘 넌 나의 보고 싶은 사람 이만 줄일게 안녕 번호 나의 유일한 번호 나의 유일한 번호 나의 유일한 번호 나의 유일한 번호 나의 유일한 번호 나의 유일한 번호 나의 유일한 번호 나의 유일한 번호 나의 유일한 번호 나는 여전히 너를 사랑하고 있다 나의 번호 나의 번호'

In [ ]:
sentence = text[0].split()


sentence = []
for i in text:
  text_split = i.split()
  sentence.append(text_split)

In [ ]:
a = []

for sentence in pos:
  a.append(sentence)
print(a)

[['가까이 사귀어'], ['가까이하다'], ['가다듬어'], ['가다듬어 수습하는'], ['가다듬어 수습하다'], ['가다듬어 정하다'], ['가당히'], ['가뜬하게'], ['가라앉혀 바로잡다'], ['가르치며'], ['가르침'], ['가벼운 마음으로'], ['가볍게 웃다'], ['가볍고'], ['가볍고 보드랍게'], ['가분가분'], ['가분히'], ['가뿐가뿐'], ['가뿐가뿐하다'], ['가뿐가뿐히'], ['가뿐하게'], ['가뿐하다'], ['가뿐한'], ['가뿐한 느낌'], ['가뿐한 느낌이'], ['가뿐히'], ['가장 중요한'], ['가지런하게 하는'], ['가지런하게 하다'], ['가지런하다'], ['가치'], ['가치가'], ['가치있는'], ['각별나다'], ['각별하다'], ['간직하다'], ['감각적이다'], ['감당하다'], ['감동시킬'], ['감동시킬 만한'], ['감모'], ['감싸고'], ['감정이 북받쳐'], ['강렬하게'], ['강세'], ['강인하다'], ['강하게'], ['강하게 하다'], ['강하고'], ['강하다'], ['강화하다'], ['갖추다'], ['개선하고'], ['개성'], ['개성있는'], ['개성있다'], ['갠소'], ['갸륵한'], ['갸륵한 데가'], ['갸름하다'], ['거리낌 없이'], ['거짓 없다'], ['거짓없다'], ['거침없이'], ['거침이 없는'], ['걱정이 없는'], ['건강에 긍정적인'], ['건둥건둥'], ['건둥건둥하다'], ['검소하다'], ['겉면이 매끄럽고'], ['겉모양의 아름다움과'], ['격려되다'], ['격려하다'], ['격식에 맞아'], ['견고하다'], ['견디어 내다'], ['견줄 수 없을 정도로'], ['결점이 없도록'], ['겸손하게'], ['겸손하게 이르는'], ['겸손하고'], ['경건한'], ['경험이 많고'], ['고상하고'], ['고상한 남자'], ['고상한 이야기'], ['고요하게'], ['고요하게 함'], ['고요하고'], ['고요함'], [

In [ ]:
okt = Okt()
kkma = Kkma()
komoran = Komoran()
hannanum = Hannanum()

In [ ]:
def apply_regular_expression(text):
    hangul = re.compile('[^ ㄱ-ㅣ 가-힣]')  # 한글 추출 규칙: 띄어 쓰기(1 개)를 포함한 한글
    result = hangul.sub('', text)  # 위에 설정한 "hangul"규칙을 "text"에 적용(.sub)시킴
    return result


In [ ]:
# 행복, 희망 => 1(긍정), 슬픔, 분노 => 0(부정) 으로 이진분류

def rating_to_label(label):
    if label == 0 or label == 3:
        return 1
    else:
        return 0
    
Lyrics['y'] = Lyrics['label'].apply(lambda x: rating_to_label(x))
Lyrics_pos = Lyrics[Lyrics['y'] == 1]
Lyrics_neg = Lyrics[Lyrics['y'] == 0]

Lyrics_pos

,Lyrics,label,y
0,아침에 일어나 너에게 짧은 인사를 보낸다 아무리 멀리 떨어져 있어도 나는 널 생각하...,0,1
1,라라랄라 라라랄라 이어 이어 네가 맛있었다고 했던 살구빛 샐러드 맛있어서 또 먹고 ...,0,1
2,자꾸 마주치는 네 눈빛에 왠지 어색해 대체 왜 이래 고개를 숙여봐도 너의 발끝에 맘...,0,1
3,어깨동무까진 괜찮아 그 치마 손잡는 건 조금 위험해 잘 자란 메세 진 괜찮아그래도 ...,0,1
4,햇살이 가득 그댈 향한 가벼운 발걸음 콧노래 부르며 쇼윈도에 비친 내 모습을 한번 ...,0,1
...,...,...,...
145,증명해 증명해 증명해 또 걔네들은 왕 이래 년째 믿고 가는 나의 촉 술 한 잔이 ...,3,1
146,들어오라 해 눈을 깔아 왜 비밀로 할 게네 부정과 죄어 전부 드러내 자릴 넘보네 ...,3,1
147,넌 대체 왜 그래난 전혀 이해 못 해 그냥 정리하란 말을 이해 못 해 넌 못 버텨...,3,1
148,이 공간 속에 비밀 가까움은 가면 같은 우린 만질 수 없는 걸 믿지 너도 알지 우...,3,1


In [ ]:
apply_regular_expression(Lyrics_pos['Lyrics'][0])
# apply_regular_expression(Lyrics_neg['Lyrics'][0])
# Lyrics_neg = Lyrics_neg.reset_index()
# Lyrics_neg = Lyrics_neg.drop(['index','level_0'],axis = 1)
# Lyrics_neg

'아침에 일어나 너에게 짧은 인사를 보낸다 아무리 멀리 떨어져 있어도 나는 널 생각하고 있어 늦은 밤 졸린 눈 비비며 너에게 편지를 쓸게 반가운 사진 속 그 표정만큼 아름다운 나날들이기를 찬 바람이 불어와 어깨가 움츠러들 때에도 너를 떠올린다면 씩씩한 표정 할 수 있어 알고 있니 넌 나의 두근대는 맘은 아직까지 여기에 언제까지나 좋아한단 말은 나를 웃음 짓게 해 때로는 지쳐 울어도 우 우 우 우 눈을 감으니 꼭 곁에 있는 것 같아 짠 하고 나타날 게 생각지 못한 어느 날 우연히 문득 뒤돌아 봤을 때 그때 꼭 내가 서 있을게 꼭 하고 껴안을게 가장 반가운 얼굴로 눈물이 왈칵 쏟아진다 해도 나는 환하게 웃을 거야 혹시 오늘 너에게 서러운 일이 생긴다면 오래전 그날처럼 달려가 꼭 안아주고파아름답게 빛나길 알고 있니 넌 나의 두근대는 맘은 아직까지 여기에 언제까지나 좋아한단 말은 나를 웃음 짓게 해 때로는 지쳐 울어도 눈을 감으니 꼭 곁에 있는 것 같아 기억해줘 넌 나의 보고 싶은 사람 이만 줄일게 안녕 번호 나의 유일한 번호 나의 유일한 번호 나의 유일한 번호 나의 유일한 번호 나의 유일한 번호 나의 유일한 번호 나의 유일한 번호 나의 유일한 번호 나의 유일한 번호 나는 여전히 너를 사랑하고 있다 나의 번호 나의 번호'

In [ ]:
okt = Okt()  # 명사 형태소 추출 함수
print("[OKt nouns 함수]")
print(okt.nouns(str(Lyrics_pos['Lyrics'])))


[OKt nouns 함수]
['아침', '너', '인사', '멀리', '나', '널', '생각', '라라', '라라', '네', '살구', '빛', '샐러드', '또', '자꾸', '네', '눈빛', '대체', '왜', '고개', '너', '발끝', '맘', '어깨동무', '그', '치마', '건', '조금', '진', '햇살', '가득', '그', '향', '발걸음', '콧노래', '쇼', '윈도', '내', '모습', '한번', '증명', '증명', '증명', '또', '걔네', '왕', '년', '나', '촉', '술', '잔', '해', '눈', '왜', '비밀', '게', '부정', '죄', '전부', '넌', '대체', '왜', '전혀', '이해', '못', '해', '그냥', '정리', '하란', '말', '이해', '못', '해', '넌', '못', '이', '공간', '속', '비밀', '움', '가면', '우린', '수', '걸', '너', '알', '무궁화', '꽃', '우리', '모두', '이', '꽃', '현아']


In [ ]:
# 말뭉치 생성
corpus_pos = "".join(Lyrics_pos['Lyrics'].tolist())
corpus_neg = "".join(Lyrics_neg['Lyrics'].tolist())
corpus_neg[:50]


'나는 볼 수 없던 이야기 이젠 그 얘기를 해주실래요 슬픈 표정 짓지 않아요 애써 웃으려도 '

In [ ]:
# 전체 말뭉치(corpus)에서 명사 형태소 추출
nouns_pos = okt.nouns(apply_regular_expression(corpus_pos))
nouns_neg = okt.nouns(apply_regular_expression(corpus_neg))
print(nouns_neg)

['나', '볼', '수', '이야기', '젠', '그', '얘기', '표정', '려', '게', '눈물', '그', '손', '워', '마음', '내', '걸', '바람', '마음', '시간', '또', '그', '끝', '새', '웃음', '것', '시간', '그', '향', '맘', '시간', '알', '수', '거', '내', '맘', '바람', '마음', '시간', '또', '그', '끝', '새', '웃음', '내', '위로', '간단', '치가', '이제야', '걸', '내', '손', '나', '보아', '빛', '하루', '그', '끝', '꿈', '사랑', '사랑', '사랑', '사랑', '사랑', '사랑', '사랑', '너', '나', '말', '너', '준', '지금', '아무', '수', '너', '준', '지금', '준', '지금', '아무', '수', '너', '준', '지금', '준', '지금', '준', '그', '어디', '수', '너', '나', '준', '준', '지금', '준', '그', '어디', '당신', '나', '당신', '마음', '나', '지금', '나', '이야기', '별자리', '상처', '투성이', '자리', '내', '혈액형', '또', '형', '날', '매일', '사람', '사람', '내', '목소리', '내', '마음', '길', '수도', '사람', '그림자', '마', '것', '게', '내겐', '그냥', '대로', '거울', '땐', '때', '마', '밤하늘', '난', '내', '나', '별자리', '아무', '나', '빛', '엄마', '방', '불빛', '이제', '그', '고단', '하루', '보고서', '잠', '그냥', '마음', '진', '맘', '마음', '것', '게', '내겐', '그냥', '대로', '거울', '땐', '때', '마', '밤하늘', '난', '내', '나', '별자리', '아무', '나', '빛', '그땐', '내', '잘못', '줄', '날', '야단',

In [ ]:
# 빈도 탐색
counter_pos = Counter(nouns_pos)
counter_neg = Counter(nouns_neg)
counter_neg.most_common(20)

[('내', 313),
 ('나', 267),
 ('너', 266),
 ('날', 212),
 ('사랑', 166),
 ('그', 157),
 ('수', 146),
 ('난', 142),
 ('말', 139),
 ('더', 134),
 ('우리', 124),
 ('것', 119),
 ('그대', 112),
 ('오', 106),
 ('네', 96),
 ('꿈', 90),
 ('게', 85),
 ('이', 82),
 ('마음', 80),
 ('걸', 80)]

In [ ]:
nouns_pos

In [ ]:
# 한글자 명사 제거
available_counter_pos = Counter({x: counter_pos[x] for x in counter_pos if len(x) > 1})
available_counter_neg = Counter({x: counter_neg[x] for x in counter_neg if len(x) > 1})
# 명사 개수 세기


stopwords = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt").values.tolist()
stopwords[:10]

# 추가할 불용어

lyric_stopwords = pd.read_csv('/content/Lyrics_word.txt').values.tolist()
for word in lyric_stopwords:
    stopwords.append(word)


from sklearn.feature_extraction.text import CountVectorizer

def text_cleaning(text):
    hangul = re.compile('[^ ㄱ-ㅣ 가-힣]')  # 정규 표현식 처리
    result = hangul.sub('', text)
    okt = Okt()  # 형태소 추출
    nouns = okt.nouns(result)
    nouns = [x for x in nouns if len(x) > 1]  # 한글자 키워드 제거
    nouns = [x for x in nouns if x not in stopwords]  # 불용어 제거
    return nouns

vect = CountVectorizer(tokenizer = lambda x: text_cleaning(x))

bow_vect_pos = vect.fit_transform(Lyrics_pos['Lyrics'].tolist())
bow_vect_neg = vect.fit_transform(Lyrics_neg['Lyrics'].tolist())

word_list = vect.get_feature_names()

count_list_pos = bow_vect_pos.toarray().sum(axis=0)
count_list_neg = bow_vect_neg.toarray().sum(axis=0)




In [ ]:
print(len(count_list_pos)) #불용어 제거 된 긍정 가사 단어 개수
print(len(count_list_neg)) #불용어 제거 된 부정 가사 단어 개수


1638
996


In [ ]:
available_counter_pos

Counter({'아침': 11,
         '인사': 1,
         '멀리': 10,
         '생각': 63,
         '비비': 2,
         '편지': 2,
         '사진': 3,
         '표정': 9,
         '나날': 1,
         '이기': 1,
         '바람': 14,
         '어깨': 12,
         '근대': 5,
         '여기': 24,
         '언제': 16,
         '웃음': 20,
         '때로는': 4,
         '어도': 2,
         '문득': 4,
         '그때': 4,
         '가장': 15,
         '얼굴': 27,
         '눈물': 11,
         '왈칵': 1,
         '해도': 9,
         '혹시': 5,
         '오늘': 42,
         '그날': 13,
         '기억': 24,
         '보고': 43,
         '사람': 69,
         '안녕': 5,
         '번호': 25,
         '사랑': 310,
         '라라': 26,
         '살구': 4,
         '샐러드': 7,
         '행복': 9,
         '달력': 1,
         '기념일': 7,
         '나나': 16,
         '우후': 3,
         '멜로디언': 3,
         '소리': 57,
         '멜로디': 13,
         '라라라': 3,
         '조그만': 1,
         '고양이': 4,
         '자꾸': 34,
         '눈빛': 5,
         '대체': 10,
         '고개': 10,
         '발끝': 4,
         '내

In [ ]:
aa = ""
total = Lyrics_pos['Lyrics']
for k in range(0, len(total)):
    characters = "ㅋㄱ"
    try:
        aa = ''.join( x for x in total[k] if x not in characters)
        data = kkma.pos(aa, join=True)
    except:
        data = kkma.pos(total[k], join=True)
    print(data)
    count_POS = 0
    count_NEG = 0
    i = 0
    while i < len(data):
        for j in range(0, len(kosac)):
            if data[i] == kosac.iloc[j, 0]:
                try:
                    if data[i+1] == kosac.iloc[j+1, 0].split(";")[1]:
                        if data[i+2] == kosac.iloc[j+2, 0].split(";")[2]:
                            # print(f"단어: {kosac.iloc[j+2, 0]}, 긍부정:{kosac.loc[j+2, 'max.value']}")
                            i += 3
                            if kosac.loc[j+2, 'max.value'] == 'POS': count_POS += 1
                            elif kosac.loc[j+2, 'max.value'] == 'NEG': count_NEG += 1
                            break
                        else:
                            # print(f"단어: {kosac.iloc[j+1, 0]}, 긍부정:{kosac.loc[j+1, 'max.value']}")
                            i += 2
                            if kosac.loc[j+1, 'max.value'] == 'POS': count_POS += 1
                            elif kosac.loc[j+1, 'max.value'] == 'NEG': count_NEG += 1
                            break
                    else:
                        # print(f"단어: {kosac.iloc[j, 0]}, 긍부정:{kosac.loc[j, 'max.value']}")
                        i += 1
                        if kosac.loc[j, 'max.value'] == 'POS': count_POS += 1
                        elif kosac.loc[j, 'max.value'] == 'NEG': count_NEG += 1
                        break
                except:
                  pass
                    # print("")
            elif data[i] != kosac.iloc[j, 0] and j == len(kosac)-1:
                i += 1
    print(f"긍정개수: {count_POS}, 부정개수: {count_NEG}")
    Lyrics_pos.iloc[k, 2] = count_POS - count_NEG
    print(Lyrics_pos.iloc[k, 2])

['아침/NNG', '에/JKM', '일어/NNG', '나/JX', '너/NP', '에게/JKM', '짧/VA', '은/ETD', '인사/NNG', '를/JKO', '보내/VV', 'ㄴ다/ECS', '아무리/MAG', '멀리/MAG', '떨어지/VV', '어/ECS', '있/VXV', '어도/ECD', '나/NP', '는/JX', '넣/VV', 'ㄹ/ETD', '생각/NNG', '하/XSV', '고/ECE', '있/VXV', '어/ECD', '늦/VA', '은/ETD', '밤/NNG', '졸리/VV', 'ㄴ/ETD', '눈/NNG', '비비/VV', '며/ECE', '너/NP', '에게/JKM', '편지/NNG', '를/JKO', '쓸/VV', '게/ECD', '반갑/VA', 'ㄴ/ETD', '사진/NNG', '속/NNG', '그/MDT', '표정/NNG', '만큼/JX', '아름답/VA', 'ㄴ/ETD', '나날/NNG', '들/XSN', '이/JKS', '기르/VV', 'ㄹ/ETD', '찬/NNG', '바람/NNG', '이/JKS', '불어오/VV', '아/ECS', '어깨/NNG', '가/JKS', '움츠러들/VV', 'ㄹ/ETD', '때/NNG', '에/JKM', '도/JX', '너/NP', '를/JKO', '떠올리/VV', 'ㄴ다면/ECE', '씩씩/XR', '하/XSA', 'ㄴ/ETD', '표정/NNG', '하/VV', 'ㄹ/ETD', '수/NNB', '있/VV', '어/ECD', '알/VV', '고/ECE', '있/VXV', '니/EFQ', '너/NP', '는/JX', '나의/NNG', '두근대/VV', '는/ETD', '맘/NNG', '은/JX', '아직/MAG', '까/VV', '지/ECD', '여기/NP', '에/JKM', '언제/NP', '까지/JX', '나/JX', '좋아하/VV', 'ㄴ단/ETD', '말/NNG', '은/JX', '나르/VV', 'ㄹ/ETD', '웃음/NNG', '짓/VV', '게/ECD', '하/VV', '어/ECS',